In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
from conllu import parse_incr, TokenList
from enum import Enum
from typing import Iterator, List, Dict, Tuple

In [ ]:
class SmoothingStrategy(Enum):
    uniform = 1
    always_other = 2
    other_and_misc = 3
    one_shot_word = 4

smoothing_strategy = None
laplace_correction = np.finfo(float).tiny
lang = 'eng'
train_set = open(f"data/{lang}/train.conllu", "r", encoding="utf-8")
test_set = open(f"data/{lang}/test.conllu", "r", encoding="utf-8")
val_set = open(f"data/{lang}/val.conllu", "r", encoding="utf-8")

tags = ['START', 'O', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'B-LOC', 'B-MISC', 'B-PER', 'B-ORG']

## Smoothing strategies

In [ ]:
def unknown_word_emission(smoothing_strategy: Enum, tag: str) -> float:
    if smoothing_strategy == SmoothingStrategy.uniform:
        return 1 / len(tags)
    elif smoothing_strategy == SmoothingStrategy.always_other:
        if tag == 'O':
            return 1
    elif smoothing_strategy == SmoothingStrategy.other_and_misc:
        if tag == 'O' or tag == 'B-MISC':
            return 0.5
    elif smoothing_strategy == SmoothingStrategy.one_shot_word:
        return one_shot_distrib[tag]
    return 0

## Matrix Functions

In [ ]:
def compute_transition_matrix(tags: List[str], train_set: Iterator[TokenList]) -> np.array:
    transition_matrix = np.zeros((len(tags), len(tags)), dtype=float)

    tag_counter = defaultdict(int)
    transition_counter = defaultdict(int)

    for sentence in parse_incr(train_set):
        # count first tag of sentence and match it with 'START' artificial tag
        first_tag = sentence[0]['lemma']
        tag_counter['START'] += 1
        transition_counter[('START', first_tag)] += 1

        # count middle token pairs
        for (token_a, token_b) in zip(sentence, sentence[1:]):
            tag_counter[token_a['lemma']] += 1
            transition_counter[(token_a['lemma'], token_b['lemma'])] += 1

        # count last tag of sentence
        tag_counter[sentence[-1]['lemma']] += 1

    for i, t1 in enumerate(tags):
        for j, t2 in enumerate(tags):
            if tag_counter[t1] > 0:  # if tag occurs at least once
                transition_matrix[i][j] = transition_counter[(t1, t2)] / tag_counter[t1]  # compute transition probability

    train_set.seek(0)
    return transition_matrix


def compute_emission_probabilities(train_set: Iterator[TokenList]) -> Dict[str, float]:
    word_tag_counter = defaultdict(int)
    tag_counter = defaultdict(int)
    word_counter = defaultdict(int)

    for sentence in parse_incr(train_set):
        for token in sentence:                
            word_tag_counter[(token['form'], token['lemma'])] += 1
            tag_counter[token['lemma']] += 1

    emission_probabilities = {(word, tag): count / tag_counter[tag] for (word, tag), count in word_tag_counter.items()}  # compute emission probability
    train_set.seek(0)
    return emission_probabilities


def compute_emission_matrix(tags: List[str], words: List[str], emission_probabilities: [Dict[str, float]]) -> np.array:
    emission_matrix = np.zeros((len(tags), len(words)), dtype=float)
    for i, tag in enumerate(tags):
        for j, word in enumerate(words):
            emission_matrix[i, j] = emission_probabilities.get((word, tag), unknown_word_emission(smoothing_strategy, tag))

    return emission_matrix


def compute_one_shot_distrib(val_set: Iterator[TokenList]) -> Dict[str, float]:
    word_tag_counter = defaultdict(int)
    
    for sentence in parse_incr(val_set):
        for token in sentence:                
            word_tag_counter[(token['form'], token['lemma'])] += 1
    
    one_shot_distrib = defaultdict(int)
    if smoothing_strategy == SmoothingStrategy.one_shot_word:
        for (word, tag), count in word_tag_counter.items():
            if count == 1:
                one_shot_distrib[tag] += 1
        total_count = sum(one_shot_distrib.values())

        for tag, count in one_shot_distrib.items():
            one_shot_distrib[tag] = count / total_count
    
    return one_shot_distrib


## Learning

In [ ]:
emission_probabilities = compute_emission_probabilities(train_set)
transition_matrix = np.log(compute_transition_matrix(tags, train_set) + laplace_correction)
Π = transition_matrix[0, 1:]

if smoothing_strategy == SmoothingStrategy.one_shot_word:
    one_shot_distrib = compute_one_shot_distrib(val_set)

tags.remove('START')
transition_matrix = transition_matrix[1:, 1:]

train_set.close()

In [ ]:
pd.DataFrame(transition_matrix, index=tags, columns=tags)

## Definition of Viterbi

In [ ]:
# TODO Need to use logarithm to compute argmax probability in order to improve performance
def argmax(viterbi_matrix: np.array, Tm: np.array, j: int, i: int) -> Tuple[int, float]:
    max_prob = np.NINF
    max_index = 0
    for index, prob in enumerate(viterbi_matrix[:, i-1]):
        prob += Tm[index, j]
        if prob > max_prob:
            max_index, max_prob = index, prob
    
    return max_index, max_prob


def viterbi(words: List[str], tags: List[str], Π: np.array, Tm: np.array, Em: np.array) -> List[str]:
    W = len(words)
    T = len(tags)

    viterbi_matrix = np.full((T, W), np.NINF, dtype=float)
    backpointer = np.empty((T, W), dtype=int)

    # compute first word initial probability for each tag
    viterbi_matrix[:, 0] = [emission + initial_p for emission, initial_p in zip(Em[:, 0], Π)]

    # compute probabilities and fill backpointer for the rest of matrix
    for i in range(1, W):
        for j in range(T):
            k, value = argmax(viterbi_matrix, Tm, j, i)
            viterbi_matrix[j, i] = value + Em[j, i]
            backpointer[j, i] = k

    # get tag index k of last column with highest probability
    last_column = list(viterbi_matrix[:, -1])
    max_value = max(last_column)
    k = last_column.index(max_value)

    # get best path walking through backpointer
    best_path = list()
    for i in range(W-1, -1, -1):
        best_path.append(tags[k])
        k = backpointer[k, i]
    
    best_path.reverse()
    return best_path

## Evaluation

In [ ]:
total_words = 0
correctly_classified = 0
for sentence in parse_incr(test_set):
    total_words += len(sentence)
    correct_tags = [token['lemma'] for token in sentence]
    words = np.array([token['form'] for token in sentence])

    emission_matrix = np.log(compute_emission_matrix(tags, words, emission_probabilities) + laplace_correction)
    result_tags = viterbi(words, tags, Π, transition_matrix, emission_matrix)
    
    correctly_classified += sum(x == y for x, y in zip(correct_tags, result_tags))

# TODO precision = tp / (tp + fp)
# TODO recall = tp / (tp + fn)
print(f'Total words: {total_words}')
print(f'Correctly tagged: {correctly_classified}')
print(f'Accuracy: {round(correctly_classified / total_words * 100, 2)}%')